In [ ]:
# QuickBooks API Data Extraction Notebook

This notebook allows you to query the QuickBooks API directly and extract all raw data.

## Setup Instructions:
1. First run `python qb_oauth_helper.py` to get your tokens
2. Copy the tokens from `qb_tokens.txt` into the authentication cell below
3. Run all cells to extract your QuickBooks data
4. Export to CSV/Excel as needed


In [ ]:
# Import required libraries
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
from datetime import datetime
import time

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully!")


In [ ]:
# QuickBooks API Configuration
QB_SANDBOX = os.getenv('QB_SANDBOX', 'True').lower() == 'true'

# API Base URLs
if QB_SANDBOX:
    QB_BASE_URL = "https://sandbox-quickbooks.api.intuit.com"
else:
    QB_BASE_URL = "https://quickbooks.api.intuit.com"

print(f"🔧 Configuration:")
print(f"   Environment: {'🧪 Sandbox' if QB_SANDBOX else '🚀 Production'}")
print(f"   Base URL: {QB_BASE_URL}")


In [ ]:
## Authentication

**Step 1: Get your tokens using the OAuth helper script**

Before running this notebook, you need to get your QuickBooks API tokens. Run this command in your terminal:

```bash
python qb_oauth_helper.py
```

This will:
1. Start a temporary web server
2. Open your browser for QuickBooks authorization
3. Save your tokens to `qb_tokens.txt`

**Step 2: Enter your tokens below**


In [ ]:
# Paste your tokens here (from qb_tokens.txt)
ACCESS_TOKEN = ""  # Paste your access token here
COMPANY_ID = ""    # Paste your company ID here

# Validate tokens
if ACCESS_TOKEN and COMPANY_ID:
    print("✅ Tokens provided")
    print(f"   Company ID: {COMPANY_ID}")
    print(f"   Access Token: {ACCESS_TOKEN[:20]}...")
else:
    print("⚠️  Please enter your ACCESS_TOKEN and COMPANY_ID above")
    print("   Run 'python qb_oauth_helper.py' first to get tokens")


In [ ]:
# Core API function
def make_qb_api_call(query, company_id=None, access_token=None):
    """Make a QuickBooks API call"""
    
    if not company_id:
        company_id = COMPANY_ID
    if not access_token:
        access_token = ACCESS_TOKEN
        
    if not company_id or not access_token:
        return {"error": "Missing company_id or access_token"}
    
    url = f"{QB_BASE_URL}/v3/company/{company_id}/query"
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept': 'application/json'
    }
    
    params = {
        'query': query,
        'minorversion': '69'
    }
    
    try:
        print(f"🔍 Query: {query}")
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            print(f"✅ Success: {response.status_code}")
            return data
        else:
            error_msg = f"Error {response.status_code}: {response.text}"
            print(f"❌ {error_msg}")
            return {"error": error_msg}
            
    except Exception as e:
        error_msg = f"Exception: {str(e)}"
        print(f"❌ {error_msg}")
        return {"error": error_msg}

# Test the API connection
if ACCESS_TOKEN and COMPANY_ID:
    print("🧪 Testing API connection...")
    test_result = make_qb_api_call("SELECT COUNT(*) FROM CompanyInfo")
    if "error" not in test_result:
        print("✅ API connection successful!")
    else:
        print(f"❌ API connection failed: {test_result['error']}")
else:
    print("⚠️  No tokens available for testing")


In [ ]:
# Available QuickBooks entities
QB_ENTITIES = [
    "Customer", "Vendor", "Employee", "Item", "Account", "Class", "Department",
    "Invoice", "Bill", "Payment", "BillPayment", "JournalEntry", "Deposit",
    "Purchase", "Expense", "Transfer", "RefundReceipt", "CreditMemo", 
    "SalesReceipt", "Estimate", "PurchaseOrder", "CompanyInfo", "Preferences",
    "TaxCode", "TaxRate", "PaymentMethod", "Term"
]

def get_entity_data(entity_name, max_results=1000):
    """Get all data for a specific entity"""
    query = f"SELECT * FROM {entity_name} MAXRESULTS {max_results}"
    result = make_qb_api_call(query)
    
    if "error" in result:
        return None, result["error"]
    
    query_response = result.get("QueryResponse", {})
    entity_data = query_response.get(entity_name, [])
    
    print(f"📊 {entity_name}: {len(entity_data)} records")
    return entity_data, None

def get_entity_count(entity_name):
    """Get count of records for an entity"""
    query = f"SELECT COUNT(*) FROM {entity_name}"
    result = make_qb_api_call(query)
    
    if "error" in result:
        return 0, result["error"]
    
    query_response = result.get("QueryResponse", {})
    total_count = query_response.get("totalCount", 0)
    
    return total_count, None

print(f"📋 Available entities: {len(QB_ENTITIES)}")
for entity in QB_ENTITIES:
    print(f"   • {entity}")


In [ ]:
# Extract data for specific entities
ENTITIES_TO_EXTRACT = [
    "Customer", "Invoice", "Payment", "Item", "JournalEntry", 
    "Deposit", "Purchase", "Expense", "Account", "Class"
]

# Dictionary to store all extracted data
extracted_data = {}

if ACCESS_TOKEN and COMPANY_ID:
    print("📥 Extracting entity data...")
    
    for entity in ENTITIES_TO_EXTRACT:
        print(f"\n🔄 Extracting {entity}...")
        data, error = get_entity_data(entity, max_results=2000)
        
        if error:
            print(f"   ❌ Error: {error}")
            extracted_data[entity] = None
        else:
            extracted_data[entity] = data
            print(f"   ✅ Extracted {len(data)} {entity} records")
        
        # Small delay to avoid rate limiting
        time.sleep(0.2)
    
    print("\n🎉 Data extraction complete!")
    
    # Summary
    total_records = sum(len(data) if data else 0 for data in extracted_data.values())
    print(f"📊 Total records extracted: {total_records:,}")
    
else:
    print("⚠️  No tokens available. Complete authentication first.")


In [ ]:
# Convert extracted data to pandas DataFrames
dataframes = {}

if extracted_data:
    print("🐼 Converting to pandas DataFrames...")
    
    for entity, data in extracted_data.items():
        if data:
            try:
                # Flatten nested JSON and create DataFrame
                df = pd.json_normalize(data)
                dataframes[entity] = df
                
                print(f"   ✅ {entity}: {len(df)} rows × {len(df.columns)} columns")
                
                # Show first few column names
                if len(df.columns) > 0:
                    cols_preview = list(df.columns)[:5]
                    if len(df.columns) > 5:
                        cols_preview.append("...")
                    print(f"      Columns: {', '.join(cols_preview)}")
                    
            except Exception as e:
                print(f"   ❌ Error converting {entity}: {str(e)}")
        else:
            print(f"   ⚪ {entity}: No data")
    
    print(f"\n📊 Created {len(dataframes)} DataFrames")
else:
    print("⚠️  No extracted data available")


In [ ]:
# Display sample data from each DataFrame
if dataframes:
    for entity, df in dataframes.items():
        if len(df) > 0:
            print(f"\n📋 {entity} Sample Data:")
            print(f"   Shape: {df.shape}")
            print(f"   Columns: {list(df.columns)[:10]}{'...' if len(df.columns) > 10 else ''}")
            
            # Display first few rows
            display(df.head(3))
            
            print("-" * 80)
else:
    print("⚠️  No DataFrames available")


In [ ]:
# Export functions
def export_to_csv(entity_name, df, filename=None):
    """Export DataFrame to CSV"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"qb_{entity_name.lower()}_{timestamp}.csv"
    
    df.to_csv(filename, index=False)
    print(f"✅ Exported {entity_name} to {filename} ({len(df)} rows)")
    return filename

def export_to_excel(dataframes_dict, filename=None):
    """Export all DataFrames to Excel with separate sheets"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"quickbooks_data_{timestamp}.xlsx"
    
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        for entity, df in dataframes_dict.items():
            if len(df) > 0:
                # Excel sheet names have a 31 character limit
                sheet_name = entity[:31]
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"   ✅ Added {entity} sheet ({len(df)} rows)")
    
    print(f"📊 Exported all data to {filename}")
    return filename

# Export options
if dataframes:
    print("💾 Export Options:")
    print("   Uncomment the lines below to export your data:")
    print()
    print("# Option 1: Export all to CSV")
    print("# for entity, df in dataframes.items():")
    print("#     if len(df) > 0:")
    print("#         export_to_csv(entity, df)")
    print()
    print("# Option 2: Export all to Excel")
    print("# excel_file = export_to_excel(dataframes)")
    print()
    print("# Option 3: Export specific entities")
    print("# entities_to_export = ['Customer', 'Invoice', 'Payment']")
    print("# for entity in entities_to_export:")
    print("#     if entity in dataframes and len(dataframes[entity]) > 0:")
    print("#         export_to_csv(entity, dataframes[entity])")
    
else:
    print("⚠️  No data available for export")


In [ ]:
## Custom Queries

Use this section to run custom QuickBooks queries:


In [ ]:
# Custom query examples
def run_custom_query(query):
    """Run a custom QuickBooks query"""
    result = make_qb_api_call(query)
    
    if "error" in result:
        print(f"❌ Query failed: {result['error']}")
        return None
    
    # Try to extract data from various possible response formats
    query_response = result.get("QueryResponse", {})
    
    # Find the data in the response
    data = None
    for key, value in query_response.items():
        if isinstance(value, list) and len(value) > 0:
            data = value
            break
    
    if data:
        df = pd.json_normalize(data)
        print(f"✅ Query returned {len(df)} rows × {len(df.columns)} columns")
        return df
    else:
        print("⚪ Query returned no data")
        return pd.DataFrame()

# Example queries (uncomment to run)
if ACCESS_TOKEN and COMPANY_ID:
    print("🔍 Custom Query Examples:")
    print("   Uncomment any of these to run:")
    print()
    print("# Get active customers only")
    print("# active_customers = run_custom_query('SELECT * FROM Customer WHERE Active = true MAXRESULTS 100')")
    print("# if active_customers is not None and len(active_customers) > 0:")
    print("#     display(active_customers.head())")
    print()
    print("# Get recent invoices")
    print("# recent_invoices = run_custom_query(\"SELECT * FROM Invoice WHERE TxnDate >= '2024-01-01' MAXRESULTS 50\")")
    print("# if recent_invoices is not None and len(recent_invoices) > 0:")
    print("#     display(recent_invoices.head())")
    print()
    print("# Get inventory items")
    print("# inventory_items = run_custom_query(\"SELECT * FROM Item WHERE Type = 'Inventory' MAXRESULTS 100\")")
    print("# if inventory_items is not None and len(inventory_items) > 0:")
    print("#     display(inventory_items.head())")
    
else:
    print("⚠️  No tokens available for custom queries")


In [ ]:
## Summary

This notebook provides you with:

1. **OAuth Helper**: Use `python qb_oauth_helper.py` to get tokens
2. **Data Extraction**: Functions to get all raw data from QuickBooks
3. **Data Analysis**: Convert to pandas DataFrames for analysis
4. **Export Options**: Save to CSV or Excel
5. **Custom Queries**: Run any QuickBooks query you need

### Next Steps:
1. Run `python qb_oauth_helper.py` to get your tokens
2. Copy tokens from `qb_tokens.txt` into cell 4
3. Run all cells to extract your data
4. Uncomment export code to save your data
5. Use custom queries for specific data needs
6. Analyze the data using pandas functions

### Available Data:
- **Customers**: Customer information, contacts, addresses
- **Invoices**: Sales invoices, line items, amounts
- **Payments**: Customer payments, methods, dates
- **Items**: Products/services, pricing, inventory
- **Journal Entries**: Manual accounting entries
- **Deposits**: Bank deposits and details
- **Purchases**: Purchase transactions
- **Expenses**: Expense transactions
- **Accounts**: Chart of accounts
- **Classes**: QuickBooks classes for tracking

**Happy analyzing! 🎉**


In [ ]:
# QuickBooks API Data Extraction Notebook

This notebook allows you to query the QuickBooks API directly and extract all raw data.

## Setup Instructions:
1. Make sure your `.env` file has the correct QB credentials
2. Run the authentication section first
3. Use the data extraction functions to get raw data
4. Export data to CSV/Excel as needed


In [1]:
# Import required libraries
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
import base64
from urllib.parse import urlencode, quote_plus
import webbrowser
from datetime import datetime
import time

# Load environment variables
load_dotenv()

print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


In [2]:
# QuickBooks API Configuration
QB_CLIENT_ID = os.getenv('QB_CLIENT_ID')
QB_CLIENT_SECRET = os.getenv('QB_CLIENT_SECRET')
QB_SANDBOX = os.getenv('QB_SANDBOX', 'True').lower() == 'true'
QB_REDIRECT_URI = os.getenv('QB_REDIRECT_URI', 'http://localhost:8888/callback')

# API Base URLs
if QB_SANDBOX:
    QB_BASE_URL = "https://sandbox-quickbooks.api.intuit.com"
    QB_DISCOVERY_URL = "https://appcenter.intuit.com/connect/oauth2"
else:
    QB_BASE_URL = "https://quickbooks.api.intuit.com"
    QB_DISCOVERY_URL = "https://appcenter.intuit.com/connect/oauth2"

print(f"🔧 Configuration:")
print(f"   Client ID: {'✅ Set' if QB_CLIENT_ID else '❌ Missing'}")
print(f"   Client Secret: {'✅ Set' if QB_CLIENT_SECRET else '❌ Missing'}")
print(f"   Environment: {'🧪 Sandbox' if QB_SANDBOX else '🚀 Production'}")
print(f"   Base URL: {QB_BASE_URL}")


🔧 Configuration:
   Client ID: ✅ Set
   Client Secret: ✅ Set
   Environment: 🚀 Production
   Base URL: https://quickbooks.api.intuit.com


In [3]:
## Authentication

**Option 1: Use existing tokens from your Flask app**
If you already have access tokens from your Flask app, enter them below:


SyntaxError: invalid syntax (993591020.py, line 3)

In [4]:
# Option 1: Manual token entry (if you have them from Flask app)
# You can get these from your Flask app session or Railway logs

ACCESS_TOKEN = ""  # Enter your access token here
COMPANY_ID = ""    # Enter your company ID here

# If you don't have tokens, leave empty and use Option 2 below
if ACCESS_TOKEN and COMPANY_ID:
    print("✅ Using manual tokens")
    print(f"   Company ID: {COMPANY_ID}")
    print(f"   Access Token: {ACCESS_TOKEN[:20]}...")
else:
    print("⚠️  No manual tokens provided. Use Option 2 for OAuth flow.")


⚠️  No manual tokens provided. Use Option 2 for OAuth flow.


In [5]:
# Core API function
def make_qb_api_call(query, company_id=None, access_token=None):
    """Make a QuickBooks API call"""
    
    if not company_id:
        company_id = COMPANY_ID
    if not access_token:
        access_token = ACCESS_TOKEN
        
    if not company_id or not access_token:
        return {"error": "Missing company_id or access_token"}
    
    url = f"{QB_BASE_URL}/v3/company/{company_id}/query"
    
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Accept': 'application/json'
    }
    
    params = {
        'query': query,
        'minorversion': '69'
    }
    
    try:
        print(f"🔍 Query: {query}")
        response = requests.get(url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            print(f"✅ Success: {response.status_code}")
            return data
        else:
            error_msg = f"Error {response.status_code}: {response.text}"
            print(f"❌ {error_msg}")
            return {"error": error_msg}
            
    except Exception as e:
        error_msg = f"Exception: {str(e)}"
        print(f"❌ {error_msg}")
        return {"error": error_msg}

# Test the API connection
if ACCESS_TOKEN and COMPANY_ID:
    print("🧪 Testing API connection...")
    test_result = make_qb_api_call("SELECT COUNT(*) FROM CompanyInfo")
    if "error" not in test_result:
        print("✅ API connection successful!")
    else:
        print(f"❌ API connection failed: {test_result['error']}")
else:
    print("⚠️  No tokens available for testing")


⚠️  No tokens available for testing


In [6]:
# Available QuickBooks entities
QB_ENTITIES = [
    "Customer", "Vendor", "Employee", "Item", "Account", "Class", "Department",
    "Invoice", "Bill", "Payment", "BillPayment", "JournalEntry", "Deposit",
    "Purchase", "Expense", "Transfer", "RefundReceipt", "CreditMemo", 
    "SalesReceipt", "Estimate", "PurchaseOrder", "CompanyInfo", "Preferences",
    "TaxCode", "TaxRate", "PaymentMethod", "Term"
]

def get_entity_data(entity_name, max_results=1000):
    """Get all data for a specific entity"""
    query = f"SELECT * FROM {entity_name} MAXRESULTS {max_results}"
    result = make_qb_api_call(query)
    
    if "error" in result:
        return None, result["error"]
    
    query_response = result.get("QueryResponse", {})
    entity_data = query_response.get(entity_name, [])
    
    print(f"📊 {entity_name}: {len(entity_data)} records")
    return entity_data, None

def get_entity_count(entity_name):
    """Get count of records for an entity"""
    query = f"SELECT COUNT(*) FROM {entity_name}"
    result = make_qb_api_call(query)
    
    if "error" in result:
        return 0, result["error"]
    
    query_response = result.get("QueryResponse", {})
    total_count = query_response.get("totalCount", 0)
    
    return total_count, None

print(f"📋 Available entities: {len(QB_ENTITIES)}")
for entity in QB_ENTITIES:
    print(f"   • {entity}")


📋 Available entities: 27
   • Customer
   • Vendor
   • Employee
   • Item
   • Account
   • Class
   • Department
   • Invoice
   • Bill
   • Payment
   • BillPayment
   • JournalEntry
   • Deposit
   • Purchase
   • Expense
   • Transfer
   • RefundReceipt
   • CreditMemo
   • SalesReceipt
   • Estimate
   • PurchaseOrder
   • CompanyInfo
   • Preferences
   • TaxCode
   • TaxRate
   • PaymentMethod
   • Term


In [7]:
# Extract data for specific entities
ENTITIES_TO_EXTRACT = [
    "Customer", "Invoice", "Payment", "Item", "JournalEntry", 
    "Deposit", "Purchase", "Expense", "Account", "Class"
]

# Dictionary to store all extracted data
extracted_data = {}

if ACCESS_TOKEN and COMPANY_ID:
    print("📥 Extracting entity data...")
    
    for entity in ENTITIES_TO_EXTRACT:
        print(f"\n🔄 Extracting {entity}...")
        data, error = get_entity_data(entity, max_results=2000)
        
        if error:
            print(f"   ❌ Error: {error}")
            extracted_data[entity] = None
        else:
            extracted_data[entity] = data
            print(f"   ✅ Extracted {len(data)} {entity} records")
        
        # Small delay to avoid rate limiting
        time.sleep(0.2)
    
    print("\n🎉 Data extraction complete!")
    
    # Summary
    total_records = sum(len(data) if data else 0 for data in extracted_data.values())
    print(f"📊 Total records extracted: {total_records:,}")
    
else:
    print("⚠️  No tokens available. Complete authentication first.")


⚠️  No tokens available. Complete authentication first.


In [8]:
# Convert extracted data to pandas DataFrames
dataframes = {}

if extracted_data:
    print("🐼 Converting to pandas DataFrames...")
    
    for entity, data in extracted_data.items():
        if data:
            try:
                # Flatten nested JSON and create DataFrame
                df = pd.json_normalize(data)
                dataframes[entity] = df
                
                print(f"   ✅ {entity}: {len(df)} rows × {len(df.columns)} columns")
                
                # Show first few column names
                if len(df.columns) > 0:
                    cols_preview = list(df.columns)[:5]
                    if len(df.columns) > 5:
                        cols_preview.append("...")
                    print(f"      Columns: {', '.join(cols_preview)}")
                    
            except Exception as e:
                print(f"   ❌ Error converting {entity}: {str(e)}")
        else:
            print(f"   ⚪ {entity}: No data")
    
    print(f"\n📊 Created {len(dataframes)} DataFrames")
else:
    print("⚠️  No extracted data available")


⚠️  No extracted data available


In [9]:
# Export functions
def export_to_csv(entity_name, df, filename=None):
    """Export DataFrame to CSV"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"qb_{entity_name.lower()}_{timestamp}.csv"
    
    df.to_csv(filename, index=False)
    print(f"✅ Exported {entity_name} to {filename} ({len(df)} rows)")
    return filename

def export_to_excel(dataframes_dict, filename=None):
    """Export all DataFrames to Excel with separate sheets"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"quickbooks_data_{timestamp}.xlsx"
    
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        for entity, df in dataframes_dict.items():
            if len(df) > 0:
                # Excel sheet names have a 31 character limit
                sheet_name = entity[:31]
                df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"   ✅ Added {entity} sheet ({len(df)} rows)")
    
    print(f"📊 Exported all data to {filename}")
    return filename

# Export options
if dataframes:
    print("💾 Export Options:")
    print("   1. Export individual entities to CSV")
    print("   2. Export all data to Excel")
    print("   3. Export specific entities")
    
    # Uncomment the export method you want to use:
    
    # Option 1: Export all to CSV
    # for entity, df in dataframes.items():
    #     if len(df) > 0:
    #         export_to_csv(entity, df)
    
    # Option 2: Export all to Excel
    # excel_file = export_to_excel(dataframes)
    
    print("\n💡 Uncomment the export code above to save your data!")
else:
    print("⚠️  No data available for export")


⚠️  No data available for export


In [ ]:
## Summary

This notebook provides you with:

1. **Authentication**: OAuth flow or manual token entry
2. **Data Extraction**: Functions to get all raw data from QuickBooks
3. **Data Analysis**: Convert to pandas DataFrames for analysis
4. **Export Options**: Save to CSV or Excel
5. **Custom Queries**: Run any QuickBooks query you need

### Next Steps:
- Enter your ACCESS_TOKEN and COMPANY_ID in cell 4
- Run all cells to extract data
- Uncomment export code to save your data
- Modify `ENTITIES_TO_EXTRACT` to get different data
- Use custom queries for specific data needs
- Analyze the data using pandas functions
